# Üst - Alt Farkı Skoru(Up-Down Differnce Score)

In [40]:
import pandas as pd
import cmath
import scipy.stats as st

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.float_format", lambda x: '%.5f' % x)

Öncelikle yanlış yapılan yöntemleri görelim:

In [1]:
#Yöntem-1
#######################################################
# Up-Down Diff Score = (up ratings) -(down ratings)
#######################################################

In [ ]:
#Soru: Elimde iki tane review(yorum) var. Birinci yorum 600 tane up(beğeni) almış, 400 tane down(dislike) almış.
                                         #İkinci yorum 5500 up(beğeni), 4500 down(dislike) almış. Bunlardan hangisini tercih edersiniz?
        
#Review 1: 600 up 400 down total 1000
#Review 2: 5500 up 4500 down total 10000    

In [18]:
#Formülasyon:
def score_up_down_diff(up, down):
    return up - down

In [7]:
#Birinci yorumun skoruna bakalım:
score_up_down_diff(600, 400)    

200

In [8]:
#İkinci yorumun skoruna bakalım:
score_up_down_diff(5500, 4500)  

1000

Şimdi bu sonuçlara göre hangi yorumu yukarıda tutardınız?
Review 2 gibi gözüküyor fakat birincisi %60 pozitif skor, ikincisi %55 pozitif skor yani birinciyi seçmek daha iyi olacak. Yani yüzdelikten dolayı birinci review  kazanmaktadır. Dolaysıyla up down yöntemi ile iki tane değer üzerinden böyle bir fark işlemi yapmak yeteri kadar doğru olmamaktadır.

In [10]:
#Yöntem-2
#######################################################
# Score = Average rating = (up ratings) / (all ratings)
#######################################################
#Yani up bölü bütün rating puanları dediğimizde elimizde pozitif ratinglerin yani begendim denilenlerin oranı olacaktır.

In [21]:
#Formülasyon:
def score_average_rating(up, down):
    if up + down ==0:
        return 0
    return up / (up + down)

In [14]:
#Değerlendirelim: İlk yorum için bir skor hesabı yapalım:
score_average_rating(600, 400)       

0.6

In [15]:
#İkincisi için skor hesabı yapalım:
score_average_rating(5500, 4500)

0.55

Sonuç: Faydalılık oranı fazla çıkan ilk yorum oldu.

Başka senaryolar ile bu yöntemi deneylim bakalım:
İki tane yorumumuz var:
1) Review 1: 2 up 0 down total 2
2) Review 2: 100 up 1 down total 101    

In [16]:
#score_average_rating bakalım birinci senaryo için ne verecek?
score_average_rating(2, 0)

1.0

In [17]:
#score_average_rating bakalım ikinci senaryo için ne verecek?
score_average_rating(100, 1)

0.9900990099009901

Sonuç:
Bu sonuçlara göre iki yorumdan hangisi tercih edilir? Birinci yorum tercih edilir. Fakat gerçekten o mu kazanmalı? Hayır. Yöntem-2 de patlamış gibi görünüyor. Sayı yüksekliğini, frekans yüksekliğini göz önünde bulunduramadı.

!!!!!!!!!!
Öyle bir işlem yapmam lazım ki hem oran bilgisi hem frekans bilgisi eş zamanlı göz önünde bulundurulabilecek şekilde bir sıralama skoru elde edelim. Bu yönteme aşağıda bakalım.

In [3]:
######################################
# Wilson Lower Bound Score
######################################


Bu yöntem ikili interaction lar, etkileşimler("Like- Dislike" vb.) sonucu ortaya çıkan bütün ölçme problemlerinde bize yardımı dokunur. Daha teknik olarak bakacak olursak bir bernoulli parametresi p ile ilgili güven aralığı hesaplar, bu güven aralığının alt sınırını vlb skor olarak kabul eder.

bernoulli parametresi p nedir?
Bu bir olasılık dağılımıdır. İkili olayların olasılığını hesaplamak için kullanılır. Örneğin; bir yazı tura olayının yazı gelmesi olasılığını hesaplamak için kullanılır. Yine aynı şekilde iki sonucu olan herhangi bir olayın nasıl gerçekleşebileceği olasılığını örneğin bir dönüşüm oranı metriği gibi bir metriğe göre düşünüldüğünde yine bu dönüşüm oranına ilişkin değerlendirmenin gerçekleşmesi olasılığı gibi ikili olayların gerçekleşme olasılığını hesaplamak için kullanılan bir olasılık dağılımıdır.
Bizim bu olasılık dağılımı ile ilgili bir işimiz yok. Bu olasılık dağılımındaki p parametresi, yani bir olayın gözlenmesi durumuyla ilgili olasılıktır. "Bizim buradaki ana odağımız nedir?" 
Bir olayın gerçekleşme oşlasılığı diye ifade ettiğimiz olay nedir? Up olayıdır.
Yani up / bütün olay dediğimizde buradaki orana ilişkin bize bir güven aralığı verir.
Neden böyle bir güven aralığı, olasılıksal hesap gibi işlemlere girdik?
!!! Çok değerli bir nokta var burada. Dikkat! Bizim elimizde ilgili bütün etkileşimler yok. Yani diyelim ki bir kullanıcı bir yorum yaptı. Bu yoruma bazı etkileşimler geldi. Beğendi, beğenmedi gibi bazı etkileşimler geldi ama gelebilecek olası bütün etkileşimleri bilmiyorum. Bütün veri elimde yok ama elimde bir ÖRNEKLEM var.
Örneğin, 600 tane like, 400 tane dislike içeren bir örneklem var. 1000 tane dolaysıyla var olanların içerisinden bir up oranım var. Buradan öyle bir genelleme yapmak istiyorum ki bilimsel olsun, bunu bütün kitleye yansıtabileyim ve bana güvenilir bir referans noktası versin. Dolaysıyla bu problemi bir olasılık problemi olarak ele aldığımızda ve bu ilgilendiğimiz olayı ifade eden oran değeri üzerinden bir güven aralığı hesapladığımızda elimizde çok değerli şöyle bir bilgi olacak.

Örneğin:
up sayımız kaçtı? 600, down sayımız kaçtı 400. Burada up oranımız nedir? 0.6 dır.
Dikkat! Bu 0.6 ifadesi için bir güven aralığı hesapladığımızda şunu diyor oluruz; 0.5 ile 0.7 ifadeleri gibi örnek olarak bazı aralıklar belirleriz ve artık alimizde şu yorum olur. İstatistiksel olarak şunu diyebiliyorum, 100 kullanıcıdan 95 i bu yorumla ilgili bir etkileşim sağladığında %5 yanılma payım olmakla birlikte bu yorumun up oranı 0,5 ile 0,7 arasında olacaktır. Ne kadar değerli, ne kadar güzel bir yorum yapma imkanı elde ettik. Daha önce elimizde sadece 0.6 diye bir değer vardı ve bu bir örneklem değeriydi.
Madem bu bir örneklem ve madem benim de öyle bir skora ihtiyacım var ki bunu bütün kitleye genelleyebileyim düşüncem var. Dolaysıyla şöyle bir şey yapmış oluyorum. Bu değer(0.6) için güven aralığı hesaplıyorum. Bu güven aralığı ile ilgili artık şöyle bir yorum yapabilirim. Bu yoruma yapılacak olan etkileşimler açısından up oranım hepful oranım, like oranım istatistiksel olarak %95 güven ile 0.5 ile 0.7 arasında olacaktır diyorum. Bunu üzerinden bunutek bir skora çevirmem gerektiği için de diyorum ki, alt sınır olan 0.5 i skor olarak belirliyorum.
Böylece bütün gözlem birimleri yani buradaki yorumlar, review ler, başka bir senaryoda item lar veya product lar da olabilir. Önemi yok. Önemli olan burada binary, ikili bir değerlendirmeyle ilgilenip 
ilgilenmeme durumumuz. Dolaysıyla bunu bütün gözlem birimleri için yaptığımda her birisi için garanti bir alt skorum var. Bu garanti alt skorumu referans alarak bunları sıralayabilirim. Böylece aslında binary, ikili rate leri de aslında hesaplamış oluyorum. Yani bu bir rate hesabı olarak kullanılır ve aynı zamanda buna dayalı olarak bir skorlama yapılır. 5 yıldızlı ürünlerde bir web sitesine yansıtma kaygımız vardı. Bu kaygımız yok. Dolaysıyla hesapladığımız bu rate, arka tarafta biizm bildiğimiz her bir review için rate olacak. Ben buna göre yorumları sıralayabilirim. Bunu hemen değerlendirelim.

Like - Dislike = 600 - 400
Up Oranı = 0.6
Bu yorumun up oranı = 0.5 0.7 arasındadır.
Alt Skor olarak belirlenen 0.5 i skor olarak belirliyorum diyorum.

In [41]:
def wilson_lower_bound(up, down, confidince = 0.95):
    #wilson Lower Bound Score hesapla
    #-Bernoulli parametresi p çin hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    #-Hesaplanacak skor ürün sıralaması için kullanılır.
    #-Not:
    #Eğer skorlar 1-5 arasındaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli ye uygun hale getirilebilir.
    #Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidince) / 2)
    phat = 1.0 * up / n
    return (phat + z*z/(2*n) - z * cmath.sqrt((phat - (1 - phat) + z*z/(4*n)) / n)) / (1 + z*z /n)

In [42]:
#Fonksiyonumu kayıt ettim ve önceki senaryolara tekrar bakalım:
wilson_lower_bound(600, 400)

(0.5719391034511111+0j)

In [43]:
#Aynı durumu diğer örnek içinde gerçekleştirelim:
wilson_lower_bound(5500, 4500)    

(0.5437822554716435+0j)

Sonuç: Artık bu yönteme göre net bir şekilde söyleyebilirim ki 0.57 sonuçlu örnek daha iyidir. Evet daha fazla olan daha güvenilirdir dedik evet doğru ama güvenilirlik nereye kadar? Yani bunun marjinal etkisi erelerde bunu bilmiyorum.
Dolaysıyla WLB bize bu imkanı da sağlıyor. Normalde bu(5500, 4500)  daha iyi diyebiliriz. Evet ama oransal olarak (600, 400) bu daha iyi. Dolaysıyla bu etkiyi yakaladı. Her şey yolunda. Biz yine kullanıcı olarak daha fazla gördüğümüz bu şeye bakabiliriz. Ama bunu şu anda sayı arttığı için ölçemiyoruz. Oran olarak bu daha iyi.
Bu yorumun faydalı bulunma skorunun sonucudur bu çıktılar. 0.54 çıktılı örnek o kadar faydalı bulunmamış demek ki. Biz daha çok yorumun faydalı bulunup bulunmadığı ile ilgilendiğimiz için bu zaten bunun kadar faydalı bulunmamış sonucunu çıkarabiliriz.
Dolaysıyla, buradaki interaction ın yüksek olması bizim için (5500, 4500) bu daha değerlidir yorumunu yaptırmıyor.
Evet İlk yöntemin problemli yanını yakaladı. Bir de ikikinci yöntemde nasıl çalışacak aşağıda bunu gözlemleyelim.

In [30]:
wilson_lower_bound(2, 0)

0.09389300773884474

In [32]:
wilson_lower_bound(100, 1)

0.785233008615569

In [14]:
# Örnek Uygulama:
up = [15, 70, 14, 4, 2, 5, 8, 37, 21, 52, 28, 147, 61, 30, 23, 40, 37, 61, 54, 18, 12, 68]
down = [0, 2, 2, 2, 15, 2, 6, 5, 23, 8, 12, 2, 1, 1, 5, 1, 2, 6, 2, 8, 2, 2]
comments = pd.DataFrame({"up":up, "down": down})

Bunu değerlendirmeden önce daha önce görmüş olduğumuz bütün skorları bir getirelim.

In [37]:
#score_pos_neg_diff
comments["score_pos_neg_diff"] = comments.apply(lambda x: score_up_down_diff(x["up"],
                                                                             x["down"]), axis =1)

In [36]:
#score_average_rating
comments["score_average_rating"] = comments.apply(lambda x: score_average_rating(x["up"],
                                                                             x["down"]), axis =1)

In [44]:
#wilson_lower_bound
comments["wilson_lower_bound"] = comments.apply(lambda x: wilson_lower_bound(x["up"],
                                                                             x["down"]), axis =1)

In [48]:
comments.head(20)

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,15,0,15.0,1.000000,0.482478+0.000000j
1,70,2,68.0,0.972222,0.733698+0.000000j
2,14,2,12.0,0.875000,0.446780+0.000000j
3,4,2,2.0,0.666667,0.258953+0.000000j
4,2,15,-13.0,0.117647,0.188122-0.326307j
5,5,2,3.0,0.714286,0.278585+0.000000j
6,8,6,2.0,0.571429,0.367037+0.000000j
7,37,5,32.0,0.880952,0.603566+0.000000j
8,21,23,-2.0,0.477273,0.479098-0.041772j
9,52,8,44.0,0.866667,0.638749+0.000000j


In [ ]:
Burada bulunan sıralama bir mantık içeriyor mu bakalım:
Sıralamada problem var çünkü  11.sırada bulunan 147 ye 2 nin bı sırada olması çok mantıksız.
Biz burada wilson_lower_bound değişkenine göre azalan sırayla listeleyeceğiz. 


In [49]:
#Azalan sıraya göre yapalım:
comments.sort_values("wilson_lower_bound", ascending = False)

,up,down,score_pos_neg_diff,score_average_rating,wilson_lower_bound
11,147,2,145.0,0.986577,0.819422+0.000000j
1,70,2,68.0,0.972222,0.733698+0.000000j
21,68,2,66.0,0.971429,0.729705+0.000000j
12,61,1,60.0,0.983871,0.723221+0.000000j
18,54,2,52.0,0.964286,0.696128+0.000000j
17,61,6,55.0,0.910448,0.681224+0.000000j
15,40,1,39.0,0.975610,0.658564+0.000000j
9,52,8,44.0,0.866667,0.638749+0.000000j
16,37,2,35.0,0.948718,0.634139+0.000000j
13,30,1,29.0,0.967742,0.608260+0.000000j
